In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam

In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

In [7]:
pares = [
    ("Hello", "Olá"),
    ("Good morning", "Bom dia"),
    ("Good afternoon", "Boa tarde"),
    ("Good evening", "Boa noite"),
    ("Good night", "Boa noite"),
    ("How are you?", "Como você está?"),
    ("I'm fine, thank you.", "Estou bem, obrigado."),
    ("What's your name?", "Qual é o seu nome?"),
    ("My name is John.", "Meu nome é John."),
    ("Nice to meet you.", "Prazer em conhecê-lo."),
    ("Please", "Por favor"),
    ("Thank you", "Obrigado"),
    ("You're welcome", "De nada"),
    ("Excuse me", "Com licença"),
    ("I'm sorry", "Desculpe"),
    ("Yes", "Sim"),
    ("No", "Não"),
    ("Maybe", "Talvez"),
    ("I don't know", "Eu não sei"),
    ("I understand", "Eu entendo"),
    ("I don't understand", "Eu não entendo"),
    ("Can you help me?", "Você pode me ajudar?"),
    ("Where is the bathroom?", "Onde fica o banheiro?"),
    ("How much does it cost?", "Quanto custa?"),
    ("I would like a coffee.", "Eu gostaria de um café."),
    ("Do you speak English?", "Você fala inglês?"),
    ("I speak a little Portuguese.", "Eu falo um pouco de português."),
    ("What time is it?", "Que horas são?"),
    ("I'm hungry.", "Estou com fome."),
    ("I'm thirsty.", "Estou com sede."),
    ("I'm tired.", "Estou cansado."),
    ("I need a doctor.", "Eu preciso de um médico."),
    ("Call the police!", "Chame a polícia!"),
    ("I love you.", "Eu te amo."),
    ("Congratulations!", "Parabéns!"),
    ("Happy birthday!", "Feliz aniversário!"),
    ("Merry Christmas!", "Feliz Natal!"),
    ("Happy New Year!", "Feliz Ano Novo!"),
    ("Good luck!", "Boa sorte!"),
    ("Have a nice day!", "Tenha um bom dia!"),
    ("See you later.", "Até mais."),
    ("Goodbye", "Adeus"),
    ("What's your phone number?", "Qual é o seu número de telefone?"),
    ("Where do you live?", "Onde você mora?"),
    ("I live in São Paulo.", "Eu moro em São Paulo."),
    ("I'm from Brazil.", "Eu sou do Brasil."),
    ("What do you do?", "O que você faz?"),
    ("I'm a teacher.", "Eu sou professor."),
    ("I work in an office.", "Eu trabalho em um escritório."),
    ("I'm a student.", "Eu sou estudante."),
    ("I like to read books.", "Eu gosto de ler livros."),
    ("I enjoy listening to music.", "Eu gosto de ouvir música."),
    ("I like to travel.", "Eu gosto de viajar."),
    ("What's your favorite food?", "Qual é a sua comida favorita?"),
    ("I like pizza.", "Eu gosto de pizza."),
    ("Do you have any pets?", "Você tem animais de estimação?"),
    ("I have a dog.", "Eu tenho um cachorro."),
    ("I don't have any pets.", "Eu não tenho animais de estimação."),
    ("What's your favorite color?", "Qual é a sua cor favorita?"),
    ("My favorite color is blue.", "Minha cor favorita é azul."),
    ("Do you like sports?", "Você gosta de esportes?"),
    ("I like soccer.", "Eu gosto de futebol."),
    ("I don't like sports.", "Eu não gosto de esportes."),
    ("What did you do yesterday?", "O que você fez ontem?"),
    ("I went to the movies.", "Eu fui ao cinema."),
    ("I stayed at home.", "Eu fiquei em casa."),
    ("What are you doing now?", "O que você está fazendo agora?"),
    ("I'm studying English.", "Estou estudando inglês."),
    ("I'm watching TV.", "Estou assistindo TV."),
    ("Do you have brothers or sisters?", "Você tem irmãos ou irmãs?"),
    ("I have one brother.", "Eu tenho um irmão."),
    ("I have two sisters.", "Eu tenho duas irmãs."),
    ("I'm an only child.", "Eu sou filho único."),
    ("What time do you wake up?", "A que horas você acorda?"),
    ("I wake up at 7 a.m.", "Eu acordo às 7 da manhã."),
    ("What time do you go to bed?", "A que horas você vai dormir?"),
    ("I go to bed at 10 p.m.", "Eu vou dormir às 10 da noite."),
    ("Do you like to cook?", "Você gosta de cozinhar?"),
    ("Yes, I love cooking.", "Sim, eu adoro cozinhar."),
    ("No, I don't like to cook.", "Não, eu não gosto de cozinhar."),
    ("What's your favorite movie?", "Qual é o seu filme favorito?"),
    ("I like action movies.", "Eu gosto de filmes de ação."),
    ("I prefer comedies.", "Eu prefiro comédias."),
    ("Do you play any instruments?", "Você toca algum instrumento?"),
    ("I play the guitar.", "Eu toco violão."),
    ("I don't play any instruments.", "Eu não toco nenhum instrumento."),
    ("What's the weather like?", "Como está o tempo?"),
    ("It's sunny.", "Está ensolarado."),
    ("It's raining.", "Está chovendo."),
    ("It's cold.", "Está frio."),
    ("It's hot.", "Está quente."),
    ("Do you like reading?", "Você gosta de ler?"),
    ("Yes, I read every day.", "Sim, eu leio todos os dias."),
    ("No, I prefer watching movies.", "Não, eu prefiro assistir filmes."),
    ("What's your favorite book?", "Qual é o seu livro favorito?"),
    ("I like mystery novels.", "Eu gosto de romances de mistério."),
    ("I enjoy science fiction.", "Eu gosto de ficção científica."),
    ("Can you drive?", "Você sabe dirigir?"),
    ("Yes, I have a driver's license.", "Sim, eu tenho carteira de motorista."),
    ("No, I don't drive.", "Não, eu não dirijo."),
    ("Do you like coffee or tea?", "Você gosta de café ou chá?"),
    ("I prefer coffee.", "Eu prefiro café."),
    ("I like both.", "Eu gosto dos dois."),
    ("I don't like either.", "Eu não gosto de nenhum."),
]

In [21]:
entrada_textos = []
saida_textos = []

# <BOS> - Begin of Sentence
# <EOS> - End of Sentence

In [23]:
for par in pares: 
    #               0                       1
    # par ==> ("I don't like either.", "Eu não gosto de nenhum.")
    entrada_textos.append( par[0].lower() )
    saida_textos.append( f"<BOS> {par[1]} <EOS>")

In [27]:
entrada_tokenizer = Tokenizer()  # Textos em ingles - entrada
entrada_tokenizer.fit_on_texts(entrada_textos)
entrada_vocabulario = entrada_tokenizer.word_index

In [33]:
saida_tokenizer = Tokenizer() # Textos em portugues - saida
saida_tokenizer.fit_on_texts(saida_textos)
saida_vocabulario = saida_tokenizer.word_index

In [37]:
entrada_sequencias = entrada_tokenizer.texts_to_sequences( entrada_textos )
saida_sequencias = saida_tokenizer.texts_to_sequences( saida_textos )
entrada_vocabulario_tam = len(entrada_vocabulario) + 1
saida_vocabulario_tam = len(saida_vocabulario) + 1

In [43]:
entrada_maximo_palavras = 0
for seq in entrada_sequencias:
    if len(seq) > entrada_maximo_palavras:
        entrada_maximo_palavras = len(seq)
entrada_maximo_palavras

8

In [45]:
saida_maximo_palavras = 0
for seq in saida_sequencias:
    if len(seq) > saida_maximo_palavras:
        saida_maximo_palavras = len(seq)
saida_maximo_palavras

9

In [47]:
entrada_padded = pad_sequences(entrada_sequencias, maxlen=entrada_maximo_palavras, padding="post")

In [49]:
entrada_padded

array([[ 62,   0,   0,   0,   0,   0,   0,   0],
       [ 13,  63,   0,   0,   0,   0,   0,   0],
       [ 13,  64,   0,   0,   0,   0,   0,   0],
       [ 13,  65,   0,   0,   0,   0,   0,   0],
       [ 13,  66,   0,   0,   0,   0,   0,   0],
       [ 27,  28,   2,   0,   0,   0,   0,   0],
       [  5,  67,  29,   2,   0,   0,   0,   0],
       [ 10,  11,  30,   0,   0,   0,   0,   0],
       [ 31,  30,  16,  68,   0,   0,   0,   0],
       [ 32,   6,  69,   2,   0,   0,   0,   0],
       [ 70,   0,   0,   0,   0,   0,   0,   0],
       [ 29,   2,   0,   0,   0,   0,   0,   0],
       [ 71,  72,   0,   0,   0,   0,   0,   0],
       [ 73,  33,   0,   0,   0,   0,   0,   0],
       [  5,  74,   0,   0,   0,   0,   0,   0],
       [ 17,   0,   0,   0,   0,   0,   0,   0],
       [ 18,   0,   0,   0,   0,   0,   0,   0],
       [ 75,   0,   0,   0,   0,   0,   0,   0],
       [  1,   8,  76,   0,   0,   0,   0,   0],
       [  1,  34,   0,   0,   0,   0,   0,   0],
       [  1,   8,  3